# Step 1. Data Wrangling
---

### - Cleaning
### - Transforming
### - and Enriching the dateset
---

- Create Virtual Environment to manage dependencies
- Activate the venv in terminal
- Add the venv as a kernel to Jupyter Lab & select it
- Install and manage dependencies in this venv

In [1]:
#!pip install pandas numpy re

- Import Necessary Modules 

In [2]:
import pandas as pd
import numpy as np
import re

- Set Some Optional Pandas Settings (Optional: Not Necessary)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

- Load Dataset as Pandas DataFrame

In [4]:
file_path = "Flipkart data/Flipkart_Mobiles.csv"
df = pd.read_csv(file_path)

print(f"Initial shape: {df.shape}")
df.head()

Initial shape: (3114, 8)


,Brand,Model,Color,Memory,Storage,Rating,Selling Price,Original Price
0,OPPO,A53,Moonlight Black,4 GB,64 GB,4.5,11990,15990
1,OPPO,A53,Mint Cream,4 GB,64 GB,4.5,11990,15990
2,OPPO,A53,Moonlight Black,6 GB,128 GB,4.3,13990,17990
3,OPPO,A53,Mint Cream,6 GB,128 GB,4.3,13990,17990
4,OPPO,A53,Electric Black,4 GB,64 GB,4.5,11990,15990


- Check Column Datatypes in loaded Dataset (Can change at initial load if needed)

In [5]:
df.dtypes

Brand              object
Model              object
Color              object
Memory             object
Storage            object
Rating            float64
Selling Price       int64
Original Price      int64
dtype: object

- Standardize Column Names

In [6]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w\s]", "", regex=True)
)
df.columns

Index(['brand', 'model', 'color', 'memory', 'storage', 'rating',
       'selling_price', 'original_price'],
      dtype='object')

- Check Null Values & Drop Duplicate rows if any

In [7]:
print("Null Values:\n",end='------------\n')
print(df.isnull().sum(),end = '\n'*2)
print("Duplicate Rows:\n",end='------------\n')
print(df.duplicated().sum())

Null Values:
------------
brand               0
model               0
color               0
memory             43
storage            39
rating            144
selling_price       0
original_price      0
dtype: int64

Duplicate Rows:
------------
108


- Drop duplicate rows if any

In [8]:
df.drop_duplicates(inplace=True)
print(f"DataFrame shape after duplicates dropped: {df.shape}")

DataFrame shape after duplicates dropped: (3006, 8)


- Utility function to replace Null/ NaN Values to nan for consistency

In [9]:
def to_nan(value):
    null_values = ['', 'null', 'none', 'n/a', '-', 'unknown']
    if pd.isnull(value) or str(value).strip().lower() in null_values:
        return np.nan
    return value

- Utility function to standardize Storage & Memory Columns

In [10]:
def extract_storage_to_mb(value):
    val_str = str(value).upper()
    match = re.search(r'(\d+\.?\d*)\s*(GB|MB)', val_str)
    if match:
        num = float(match.group(1))
        unit = match.group(2)
        if unit == 'GB':
            return num * 1024  # Convert GB to MB
        else:
            return num  # Already in MB
    else:
        return np.nan

- Apply Utility functions to concerned columns

In [11]:
df['rating'] = df['rating'].apply(to_nan)
df['memory'] = df['memory'].apply(to_nan)
df['storage'] = df['storage'].apply(to_nan)

In [12]:
df['memory'] = df['memory'].apply(extract_storage_to_mb)
df['storage'] = df['storage'].apply(extract_storage_to_mb)

- Check data distribution & drop any columns where critical information is missng
- Negative Selling price is present

In [13]:
print(df.describe())

df.dropna(subset=['brand', 'model', 'selling_price', 'original_price'], inplace=True)
df = df[df['selling_price'] > 0]

             memory        storage       rating  selling_price  original_price
count   2966.000000    2953.000000  2876.000000    3006.000000     3006.000000
mean    4251.249157   88401.518794     4.236057   25842.448436    27722.581504
std     2577.310311   92888.967595     0.270684   29658.810517    31117.486618
min        2.000000       2.000000     2.300000    1000.000000     1000.000000
25%     2048.000000   32768.000000     4.100000    9978.500000    10000.000000
50%     4096.000000   65536.000000     4.300000   14999.000000    15999.000000
75%     6144.000000  131072.000000     4.400000   27499.000000    29999.000000
max    32768.000000  524288.000000     5.000000  179900.000000   189999.000000


- Create derived columns for better data interpretation

In [14]:
df['discount_amount'] = df['original_price'] - df['selling_price']
df['discount_percent'] = ((df['discount_amount'] / df['original_price']) * 100).round(2)

df['discount_percent'] = df['discount_percent'].clip(0, 100)

df['brand'] = df['brand'].str.title()
df['model'] = df['model'].str.upper().str.strip()

- Segregate Selling prices into different buckets based on percentile

In [15]:
price_percentiles = np.percentile(df['selling_price'], [0, 20, 40, 60, 80, 100])
labels = ['Entry', 'Budget', 'Mid-Range', 'Premium', 'Flagship']

df['price_segment'] = pd.cut(df['selling_price'], bins=price_percentiles, labels=labels, include_lowest=True)

- Saving percentile Bins to a seperate CSV

In [20]:
price_seg_limits = pd.DataFrame({
    'Segment': labels,
    'Upper Limit': price_percentiles[1:].astype(int)
})

# Save to CSV
price_seg_limits.to_csv("price_segment_limits.csv", index=False)

- Final Descriptive summary of data before save

In [17]:
print("Price Segments:\n",end='------------\n')
print(df['price_segment'].value_counts())

print("\nBrands:\n", end='------------\n')
print(df['brand'].value_counts())

print("\nMemory options:\n",end='------------\n')
print(df['memory'].value_counts().sort_index())

print("\nStorage options:\n", end='------------\n')
print(df['storage'].value_counts().sort_index())


Price Segments:
------------
price_segment
Mid-Range    633
Entry        602
Budget       601
Flagship     595
Premium      575
Name: count, dtype: int64

Brands:
------------
brand
Samsung         704
Apple           324
Realme          322
Oppo            257
Nokia           199
Xiaomi          198
Infinix         150
Gionee          128
Vivo            124
Asus            117
Lenovo          117
Motorola        104
Lg               99
Poco             74
Htc              55
Google Pixel     29
Iqoo              5
Name: count, dtype: int64

Memory options:
------------
memory
2.0          2
4.0         39
8.0         10
10.0         3
16.0        15
30.0         1
32.0        14
46.0         2
64.0        14
100.0        1
128.0        4
153.0        1
512.0       46
768.0        6
1024.0     191
1536.0      29
2048.0     365
3072.0     477
4096.0     849
6144.0     492
8192.0     340
12288.0     60
16384.0      2
18432.0      2
32768.0      1
Name: count, dtype: int64

Storage optio

In [18]:
df.to_csv("cleaned_mobile_data.csv", index=False)
print("Cleaned data saved.")

Cleaned data saved.


- Combo Summary using pivot

In [19]:
combo_summary = df.pivot_table(
    index=['brand', 'model'],
    values='selling_price',
    aggfunc=['count', 'mean']
).reset_index()

combo_summary.columns = ['brand', 'model', 'num_variants', 'avg_price']
combo_summary.to_csv("brand_model_summary.csv", index=False)